<a href="https://colab.research.google.com/github/yeahginny/Machine_learning/blob/main/0825_AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install autokeras

In [13]:
import numpy as np
import tensorflow as tf
import autokeras as ak

In [4]:
# 데이터 로딩
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
# 데이터 전처리: 스케일링
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [6]:
# 데이터 전처리: 차원 확장 (AutoKeras에 맞게 4D로 만듦)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [7]:
# AutoKeras 모델 생성
clf = ak.ImageClassifier(overwrite=True, max_trials=1)

In [ ]:
# 모델 훈련
history = clf.fit(x_train, y_train, epochs=10)

In [ ]:
# 모델 평가
print(clf.evaluate(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

# 모델의 구성 출력
model = clf.export_model()
model.summary()

In [ ]:
# 학습 과정 시각화
plt.figure(figsize=(12, 5))

# 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

### AutoML

In [ ]:
import h2o
from h2o.automl import H2OAutoML

# H2O 초기화
h2o.init()

In [ ]:
# 데이터 로드
data = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")

In [ ]:
# 데이터 분할
train, test = data.split_frame([0.8])

In [ ]:
# 응답 변수 및 입력 변수 설정
response = "response"
train[response] = train[response].asfactor()
test[response] = test[response].asfactor()

In [ ]:
# AutoML 설정 및 실행
aml = H2OAutoML(max_runtime_secs=120, seed=1)
aml.train(y=response, training_frame=train)

In [ ]:
# 성능 평가
lb = aml.leaderboard
print(lb.head())

In [ ]:
# 최고 성능 모델 예측
preds = aml.leader.predict(test)

### 보스턴 집값 데이터를 AutoML로 해보기

In [ ]:
import h2o
from h2o.automl import H2OAutoML

# 1. H2O 초기화
h2o.init()

# 2. 데이터 로딩
data = h2o.import_file("Boston_house.csv")

In [ ]:
# 데이터의 기본 정보 및 처음 5행을 확인
data.describe()
data.head(5)

In [ ]:
# 3. 데이터 분할
# 'medv' 열은 보스턴 주택 데이터셋의 목표 변수 (주택의 중간 가격)
response = "Target"
train, test = data.split_frame(ratios=[0.8])

In [ ]:
# 4. AutoML 실행
# max_runtime_secs은 모델 학습에 허용된 최대 시간을 지정
aml = H2OAutoML(max_runtime_secs=120, seed=1)
aml.train(y=response, training_frame=train)